## Testing for load magasinstatistikk to source database


In [2]:
import pandas as pd
import requests
from sqlalchemy import create_engine, Engine

from typing import Iterable

/tmp/ipykernel_264954/3738510298.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def load_raw_api_data(api_url: str) -> pd.DataFrame:
    """ Request GET data from url and store in dataframe """
    r = requests.get(api_url)
    df = pd.DataFrame(r.json())

    return df

### 1. Prepare areas for loading into db table "area":
### api_url='https://biapi.nve.no/magasinstatistikk/api/Magasinstatistikk/HentOmråder'

In [4]:
area_df = load_raw_api_data(
    api_url='https://biapi.nve.no/magasinstatistikk/api/Magasinstatistikk/HentOmråder'
)

area_df.head()


,land,elspot,vassdrag
0,"[{'navn': 'Norge', 'navn_langt': 'Norge', 'bes...","[{'navn': 'NO 1', 'navn_langt': 'Elspotområde ...","[{'navn': 'VASS1', 'navn_langt': 'Vassdragsomr..."


In [ ]:
def prepare_areas(area_df: pd.DataFrame) -> pd.DataFrame:
    
    elspots_df = pd.read_csv('../elspots/elspots.csv')

    def normalize_and_assign_areas(area_df: pd.DataFrame, area: str) -> pd.DataFrame:
        return pd.json_normalize(area_df[area].iloc[0]).assign(current_area=area)

    areas = ['land', 'elspot', 'vassdrag']
    result_df = (
        pd.concat(map(lambda area: normalize_and_assign_areas(area_df, area), areas), ignore_index=True)
        .rename(columns={'omrType': 'omr_type'})
    )

    return result_df

area_df = prepare_areas(area_df)

area_df

,navn,navn_langt,beskrivelse,omr_type,omrnr,current_area
0,Norge,Norge,Hele landet,NO,0,land
1,NO 1,Elspotområde 1,Øst-Norge. Omfatter østlige del av Østlandet f...,EL,1,elspot
2,NO 2,Elspotområde 2,"Sørvest-Norge. Omfatter sørlige del av Viken, ...",EL,2,elspot
3,NO 3,Elspotområde 3,Midt-Norge. Omfatter nordre og vestlige del av...,EL,3,elspot
4,NO 4,Elspotområde 4,Nord-Norge. Omfatter resten av Trøndelag og No...,EL,4,elspot
5,NO 5,Elspotområde 5,Vest-Norge. Omfatter midtre del av Vestland op...,EL,5,elspot
6,VASS1,Vassdragsområde 1,"Sørøst-Norge. Østlandet, Agder-fylkene og dele...",VASS,1,vassdrag
7,VASS2,Vassdragsområde 2,"Vest-landet. Resten av Rogaland, mesteparten a...",VASS,2,vassdrag
8,VASS3,Vassdragsområde 3,"Midt-Norge. Møre og Romsdal, Trøndelag og sørl...",VASS,3,vassdrag
9,VASS4,Vassdragsområde 4,Nord-Norge. Resten av Nordland og nordover.,VASS,4,vassdrag


In [6]:
area_df.dtypes

navn            object
navn_langt      object
beskrivelse     object
omr_type        object
omrnr            int64
current_area    object
dtype: object

### 2. Retrieve magasin statistics data from:
### api_url='https://biapi.nve.no/magasinstatistikk/api/Magasinstatistikk/HentOffentligData'
#### We do this before preparing dates to only get the necessary date data we need for the magasin statistics

In [7]:
magasin_df = load_raw_api_data(
    api_url='https://biapi.nve.no/magasinstatistikk/api/Magasinstatistikk/HentOffentligData',
)

magasin_df['dato_Id'] = pd.to_datetime(magasin_df['dato_Id'], format='%Y-%m-%d', errors='coerce')
magasin_df['neste_Publiseringsdato'] = pd.to_datetime(magasin_df['neste_Publiseringsdato'], format='%Y-%m-%dT%H:%M:%S', errors='coerce')

### 3. Prepare dates for loading into db table "dates": 

In [8]:
def prepare_dates(years: Iterable[int]) -> pd.DataFrame:
    """Create a DataFrame modeled "dates" with all date data for given years."""
     
    # Generate all dates for the given years
    date_range = pd.date_range(start=f"{min(years)}-01-01", end=f"{max(years)}-12-31")

    df = pd.DataFrame({
        "iso_dato": date_range,
        "iso_aar": date_range.year,
        "maaling_uke": date_range.isocalendar().week,
        "iso_maaned": date_range.month,
        "iso_dag": date_range.day
    })
    
    df = df.reset_index(drop=True)

    return df

iso_aar = magasin_df['iso_aar'].unique()

dato_df = prepare_dates(iso_aar)
print(f"First 5: \n {dato_df.head(5)} \n \n Last 5: \n {dato_df.tail(5)} ")
print(iso_aar)

First 5: 
     iso_dato  iso_aar  maaling_uke  iso_maaned  iso_dag
0 1995-01-01     1995       52           1        1
1 1995-01-02     1995        1           1        2
2 1995-01-03     1995        1           1        3
3 1995-01-04     1995        1           1        4
4 1995-01-05     1995        1           1        5 
 
 Last 5: 
         iso_dato  iso_aar  maaling_uke  iso_maaned  iso_dag
11318 2025-12-27     2025       52          12       27
11319 2025-12-28     2025       52          12       28
11320 2025-12-29     2025        1          12       29
11321 2025-12-30     2025        1          12       30
11322 2025-12-31     2025        1          12       31 
[2007 2015 2005 2009 1995 2018 2016 2004 2003 2022 1997 2011 2012 2023
 2000 2020 2008 2002 2017 2024 1999 2019 1996 2010 2021 2001 1998 2013
 2006 2014 2025]


### 4. Drop 'iso_aar' and 'maaling_uke' which provides unecessary memory

In [9]:
magasin_df = magasin_df.drop(columns=['iso_aar', 'maaling_uke'])

### 5. Prepare magasin statistics for loading into db table "magasinstatistikk_model":

In [10]:
magasin_df.head()

,dato_Id,omrType,omrnr,fyllingsgrad,kapasitet_TWh,fylling_TWh,neste_Publiseringsdato,fyllingsgrad_forrige_uke,endring_fyllingsgrad
0,2007-03-04,EL,5,0.416593,17.425789,7.259468,NaT,0.450818,-0.034225
1,2015-02-08,EL,5,0.485430,17.425789,8.458999,NaT,0.527302,-0.041872
2,2005-09-11,EL,5,0.875464,17.425789,15.255660,NaT,0.858146,0.017319
3,2009-02-22,EL,5,0.391005,17.425789,6.813575,NaT,0.425169,-0.034164
4,1995-05-07,EL,5,0.172278,17.425789,3.002083,NaT,0.175498,-0.003219


In [11]:
magasin_df.shape

(14139, 9)

In [12]:
magasin_df.dtypes

dato_Id                     datetime64[ns]
omrType                             object
omrnr                                int64
fyllingsgrad                       float64
kapasitet_TWh                      float64
fylling_TWh                        float64
neste_Publiseringsdato      datetime64[ns]
fyllingsgrad_forrige_uke           float64
endring_fyllingsgrad               float64
dtype: object

In [13]:
magasin_df = magasin_df.rename(
    columns={
        'dato_Id': 'dato_id',
        'omrType': 'omr_type',
        'kapasitet_TWh': 'kapasitet_twh',
        'fylling_TWh': 'fylling_twh',
        'fyllingsgrad': 'fyllingsgrad',
        'neste_Publiseringsdato': 'neste_publiseringsdato'
    }
)
magasin_df.head()

,dato_id,omr_type,omrnr,fyllingsgrad,kapasitet_twh,fylling_twh,neste_publiseringsdato,fyllingsgrad_forrige_uke,endring_fyllingsgrad
0,2007-03-04,EL,5,0.416593,17.425789,7.259468,NaT,0.450818,-0.034225
1,2015-02-08,EL,5,0.485430,17.425789,8.458999,NaT,0.527302,-0.041872
2,2005-09-11,EL,5,0.875464,17.425789,15.255660,NaT,0.858146,0.017319
3,2009-02-22,EL,5,0.391005,17.425789,6.813575,NaT,0.425169,-0.034164
4,1995-05-07,EL,5,0.172278,17.425789,3.002083,NaT,0.175498,-0.003219


In [14]:
POSTGRES_USER="postgres"
POSTGRES_PASSWORD="changethis"

POSTGRES_SERVER="localhost"
POSTGRES_PORT=5432
POSTGRES_DB="nve_db"
SRC_DB_URL=f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_SERVER}:{POSTGRES_PORT}/{POSTGRES_DB}"


In [15]:
engine = create_engine(SRC_DB_URL)

### 6. Insert dates into "dates" table in db

In [16]:
def insert_into_db(engine: Engine, *, table_name: str, df: pd.DataFrame) -> pd.DataFrame:
    """ 
    Insert data into database and return a DF with all rows from the table,
    including any new columns created by the database 
    """ 
    with engine.begin() as conn:
        df.to_sql(
            name=table_name,
            con=conn,
            if_exists='append',
            index=False
        )
        
        new_df = pd.read_sql_table( table_name=table_name,
            con=conn
        )
            
        return new_df

In [17]:
dato_dim_table_df = insert_into_db(engine, table_name='dates', df=dato_df)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "dates_iso_dato_key"
DETAIL:  Key (iso_dato)=(1995-01-01) already exists.

[SQL: INSERT INTO dates (iso_dato, iso_aar, maaling_uke, iso_maaned, iso_dag) VALUES (%(iso_dato__0)s, %(iso_aar__0)s, %(maaling_uke__0)s, %(iso_maaned__0)s, %(iso_dag__0)s), (%(iso_dato__1)s, %(iso_aar__1)s, %(maaling_uke__1)s, %(iso_maaned__1)s, %(iso_dag__1)s), (%( ... 95173 characters truncated ... 8)s), (%(iso_dato__999)s, %(iso_aar__999)s, %(maaling_uke__999)s, %(iso_maaned__999)s, %(iso_dag__999)s)]
[parameters: {'iso_dato__0': datetime.datetime(1995, 1, 1, 0, 0), 'iso_dag__0': 1, 'maaling_uke__0': 52, 'iso_aar__0': 1995, 'iso_maaned__0': 1, 'iso_dato__1': datetime.datetime(1995, 1, 2, 0, 0), 'iso_dag__1': 2, 'maaling_uke__1': 1, 'iso_aar__1': 1995, 'iso_maaned__1': 1, 'iso_dato__2': datetime.datetime(1995, 1, 3, 0, 0), 'iso_dag__2': 3, 'maaling_uke__2': 1, 'iso_aar__2': 1995, 'iso_maaned__2': 1, 'iso_dato__3'

In [ ]:
dato_dim_table_df.head()

,id,iso_dato,iso_aar,maaling_uke,iso_maaned,iso_dag
0,1,1995-01-01,1995,52,1,1
1,2,1995-01-02,1995,1,1,2
2,3,1995-01-03,1995,1,1,3
3,4,1995-01-04,1995,1,1,4
4,5,1995-01-05,1995,1,1,5


In [ ]:
dato_dim_table_df.dtypes

id                     int64
iso_dato      datetime64[ns]
iso_aar                int64
maaling_uke                int64
iso_maaned             int64
iso_dag                int64
dtype: object

### 7. Insert areas into "area" table in db

In [25]:
elspots_df = pd.read_csv('../elspots/elspots.csv')

In [ ]:
area_table_df = insert_into_db(engine, table_name='area', df=area_df)

In [ ]:
area_table_df.head()

,id,navn,navn_langt,beskrivelse,omr_type,omrnr,current_area
0,1,Norge,Norge,Hele landet,NO,0,land
1,2,NO 1,Elspotområde 1,Øst-Norge. Omfatter østlige del av Østlandet f...,EL,1,elspot
2,3,NO 2,Elspotområde 2,"Sørvest-Norge. Omfatter sørlige del av Viken, ...",EL,2,elspot
3,4,NO 3,Elspotområde 3,Midt-Norge. Omfatter nordre og vestlige del av...,EL,3,elspot
4,5,NO 4,Elspotområde 4,Nord-Norge. Omfatter resten av Trøndelag og No...,EL,4,elspot


In [ ]:
area_table_df.dtypes

id               int64
navn            object
navn_langt      object
beskrivelse     object
omr_type        object
omrnr            int64
current_area    object
dtype: object

### 8. Merge magasin with date_dim_table and area_table dataframes and adjust columns

In [ ]:
magasin_df = (
    magasin_df
    .merge(
    right=dato_dim_table_df[['id', 'iso_dato']],
    how='left',
    left_on='dato_id',
    right_on='iso_dato',
    validate='m:1')
    .drop(columns=['dato_id'])
    .rename(columns={'id': 'dato_id'})
    .merge(
        right=area_table_df[['id', 'omr_type', 'omrnr']],
        how='left',
        left_on=['omrnr', 'omr_type'],
        right_on=['omrnr', 'omr_type'],
        validate='m:1'
    )
    .rename(columns={'id': 'area_id'})
    .drop(
        columns=['iso_dato', 'omrnr', 'omr_type']
    )
)
    

In [ ]:
magasin_df.head()

,fyllingsgrad,kapasitet_twh,fylling_twh,neste_publiseringsdato,fyllingsgrad_forrige_uke,endring_fyllingsgrad,dato_id,area_id
0,0.416593,17.425789,7.259468,NaT,0.450818,-0.034225,4446,6
1,0.485430,17.425789,8.458999,NaT,0.527302,-0.041872,7344,6
2,0.875464,17.425789,15.255660,NaT,0.858146,0.017319,3907,6
3,0.391005,17.425789,6.813575,NaT,0.425169,-0.034164,5167,6
4,0.172278,17.425789,3.002083,NaT,0.175498,-0.003219,127,6


In [26]:
elspots_df.head()

,X,Y,Z,Name
0,69.565139,21.044306,0,Elspotområde 4
1,63.092310,9.020742,0,Elspotområde 3
2,60.423713,6.093197,0,Elspotområde 5
3,58.772860,7.038022,0,Elspotområde 2
4,60.901100,11.385549,0,Elspotområde 1


### 9. Insert magasin into "magasinstatistikk_model" table in db

In [27]:
magasin_table_df = insert_into_db(engine, table_name='magasinstatistikk_model', df=magasin_df)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "omr_type" of relation "magasinstatistikk_model" does not exist
LINE 1: INSERT INTO magasinstatistikk_model (dato_id, omr_type, omrn...
                                                      ^

[SQL: INSERT INTO magasinstatistikk_model (dato_id, omr_type, omrnr, fyllingsgrad, kapasitet_twh, fylling_twh, neste_publiseringsdato, fyllingsgrad_forrige_uke, endring_fyllingsgrad) VALUES (%(dato_id__0)s, %(omr_type__0)s, %(omrnr__0)s, %(fyllingsgrad__0) ... 221842 characters truncated ... , %(neste_publiseringsdato__999)s, %(fyllingsgrad_forrige_uke__999)s, %(endring_fyllingsgrad__999)s)]
[parameters: {'fyllingsgrad_forrige_uke__0': 0.45081845, 'omr_type__0': 'EL', 'fylling_twh__0': 7.259468, 'neste_publiseringsdato__0': None, 'kapasitet_twh__0': 17.425789, 'fyllingsgrad__0': 0.41659337, 'omrnr__0': 5, 'dato_id__0': datetime.datetime(2007, 3, 4, 0, 0), 'endring_fyllingsgrad__0': -0.034225076, 'fyllingsgrad_forrige_uke__1': 0.527302, 'o

### 10. Retrieve magasin statistics min max median model from:
### api_url: https://biapi.nve.no/magasinstatistikk/api/Magasinstatistikk/HentOffentligDataMinMaxMedian

In [ ]:
magasin_min_max_df = load_raw_api_data(
    api_url='https://biapi.nve.no/magasinstatistikk/api/Magasinstatistikk/HentOffentligDataMinMaxMedian'
)

In [ ]:
magasin_min_max_df.head()

,omrType,omrnr,maaling_uke,minFyllingsgrad,minFyllingTWH,medianFyllingsGrad,medianFylling_TWH,maxFyllingsgrad,maxFyllingTWH
0,EL,1,1,0.456893,2.742963,0.616703,3.702380,0.759600,4.560268
1,EL,1,2,0.422561,2.536850,0.573354,3.442137,0.698279,4.192127
2,EL,1,3,0.389071,2.335794,0.533293,3.201628,0.655389,3.934633
3,EL,1,4,0.350166,2.102222,0.489547,2.939000,0.603621,3.623846
4,EL,1,5,0.318725,1.913467,0.449146,2.696455,0.570686,3.426118


### 11. Prepare magasin statistics min max median model for loading into db

In [ ]:
magasin_min_max_df = (
    magasin_min_max_df
    .rename(columns={
        'omrType': 'omr_type',
        'minFyllingsgrad': 'min_fyllingsgrad',
        'minFyllingTWH': 'min_fylling_twh',
        'medianFyllingsGrad': 'median_fyllingsgrad',
        'medianFylling_TWH': 'median_fylling_twh',
        'maxFyllingsgrad': 'max_fyllingsgrad',
        'maxFyllingTWH': 'max_fylling_twh',
    })
    .merge(
    right=area_table_df[['id', 'omr_type', 'omrnr']],
    how='left',
    left_on=['omrnr', 'omr_type'],
    right_on=['omrnr', 'omr_type'],
    validate='m:1'
    )
    .rename(columns={'id': 'area_id'})
    .drop(
    columns=['omrnr', 'omr_type']
    )
)



In [ ]:
magasin_min_max_df.head()

,maaling_uke,min_fyllingsgrad,min_fylling_twh,median_fyllingsgrad,median_fylling_twh,max_fyllingsgrad,max_fylling_twh,area_id
0,1,0.456893,2.742963,0.616703,3.702380,0.759600,4.560268,2
1,2,0.422561,2.536850,0.573354,3.442137,0.698279,4.192127,2
2,3,0.389071,2.335794,0.533293,3.201628,0.655389,3.934633,2
3,4,0.350166,2.102222,0.489547,2.939000,0.603621,3.623846,2
4,5,0.318725,1.913467,0.449146,2.696455,0.570686,3.426118,2


### 12. Insert magasin statistics min max median into db

In [ ]:
magasin_min_max_table_df = insert_into_db(engine, table_name='magasinstatistikk_min_max_median_model', df=magasin_min_max_df)

In [ ]:
magasin_min_max_table_df.head()

,maaling_uke,min_fyllingsgrad,min_fylling_twh,median_fyllingsgrad,median_fylling_twh,max_fyllingsgrad,max_fylling_twh,area_id
0,1,0.456893,2.742963,0.616703,3.702380,0.759600,4.560268,2
1,2,0.422561,2.536850,0.573354,3.442137,0.698279,4.192127,2
2,3,0.389071,2.335794,0.533293,3.201628,0.655389,3.934633,2
3,4,0.350166,2.102222,0.489547,2.939000,0.603621,3.623846,2
4,5,0.318725,1.913467,0.449146,2.696455,0.570686,3.426118,2


# This seems ok for now